In [1]:
import torch
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import wandb

In [2]:
!python3 -m wandb login eb7b1964fb84cd81de96b2a273ecf2bb6254aeac

wandb: Appending key for api.wandb.ai to your netrc file: /home/alexeyorlov53/.netrc


In [3]:
model_name_base = 'graphormer-base-pcqm4mv1'
model_name = 'clefourrier/graphormer-base-pcqm4mv1'

In [4]:
batch_size = 2

### Upload Dataset

In [5]:
dataframe = pd.read_csv("data_10k_graph.csv")

In [6]:
print('Percentage on NaNs:')
dataframe.isna().mean()

Percentage on NaNs:


y             0.0
Smiles        0.0
ecfp1         0.0
ecfp2         0.0
ecfp3         0.0
node_feat     0.0
edge_index    0.0
edge_attr     0.0
num_nodes     0.0
dtype: float64

In [7]:
dataframe = dataframe.drop(columns=['Smiles', 'ecfp1', 'ecfp2', 'ecfp3'])

In [8]:
def preprocess_array_column(df, column):
    for row in tqdm(range(len(df))):
        str_ints = eval(df.iloc[row][column])
        df.at[row, column] = str_ints

In [9]:
# это потому что пандас думает что массив это строка изначально
preprocess_array_column(dataframe, 'node_feat')
preprocess_array_column(dataframe, 'edge_index')
preprocess_array_column(dataframe, 'edge_attr')
preprocess_array_column(dataframe, 'y')

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
dataframe[dataframe['num_nodes'] == 1]

,y,node_feat,edge_index,edge_attr,num_nodes
2991,[17.02],"[[7, 0]]","[[], []]",[],1


In [11]:
dataframe.iloc[2991] # выброс ?

y              [17.02]
node_feat     [[7, 0]]
edge_index    [[], []]
edge_attr           []
num_nodes            1
Name: 2991, dtype: object

### Create Dataloader

In [12]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(dataframe)
train_testvalid = dataset.train_test_split(test_size=0.2, seed=15)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=15)

# 10% for test, 10 for validation, 80% for train
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset

DatasetDict({
    train: Dataset({
        features: ['y', 'node_feat', 'edge_index', 'edge_attr', 'num_nodes'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['y', 'node_feat', 'edge_index', 'edge_attr', 'num_nodes'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['y', 'node_feat', 'edge_index', 'edge_attr', 'num_nodes'],
        num_rows: 1000
    })
})

In [13]:
from transformers.models.graphormer.collating_graphormer import preprocess_item, GraphormerDataCollator

# dataset_processed = dataset.map(preprocess_item, batched=False)
# data_loader = GraphormerDataCollator(on_the_fly_processing=True) # либо препроцессинг либо коллайтор с on_the_fly_processing

/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/models/graphormer/algos_graphormer.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/alexeyorlov53/.pyxbld/temp.linux-x86_64-cpython-39/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/models/graphormer/algos_graphor

In [14]:
# dataset_processed.save_to_disk('dataset_10k_graphormer_preprocessed')

In [15]:
# тут можно подгружать чтобы быстрее
from datasets import load_from_disk
dataset_processed = load_from_disk('dataset_10k_graphormer_preprocessed')

### Create DataLoader

In [2]:
import torch

In [3]:
graph_emb = torch.tensor([[1,2,3], [4,5,6]])
bert_emb = torch.tensor([[7,8,9], [10,11,12]])

In [4]:
torch.cat((graph_emb, bert_emb), dim=-1)

tensor([[ 1,  2,  3,  7,  8,  9],
        [ 4,  5,  6, 10, 11, 12]])

In [16]:
from transformers.models.graphormer.collating_graphormer import GraphormerDataCollator

class GraphormerDataCollator_():
    def __init__(self):
        self.data_collator = GraphormerDataCollator()

    def __call__(self, features):
        for mol in features:
            if mol['num_nodes'] == 1:
                features.remove(mol)
        return self.data_collator(features)

In [17]:
from torch.utils.data import DataLoader

data_collator = GraphormerDataCollator_()

train_dataloader = DataLoader(
    dataset_processed['train'], shuffle = False, batch_size = batch_size, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    dataset_processed['validation'], shuffle = False, batch_size = batch_size, collate_fn = data_collator
)

In [18]:
device = torch.device("cuda", index=4) if torch.cuda.is_available() else torch.device('cpu')

In [19]:
from transformers import GraphormerForGraphClassification

model = GraphormerForGraphClassification.from_pretrained(
    model_name, 
    num_classes=1,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
).to(device)

/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
model

GraphormerForGraphClassification(
  (encoder): GraphormerModel(
    (graph_encoder): GraphormerGraphEncoder(
      (dropout_module): Dropout(p=0.0, inplace=False)
      (graph_node_feature): GraphormerGraphNodeFeature(
        (atom_encoder): Embedding(4609, 768, padding_idx=0)
        (in_degree_encoder): Embedding(512, 768, padding_idx=0)
        (out_degree_encoder): Embedding(512, 768, padding_idx=0)
        (graph_token): Embedding(1, 768)
      )
      (graph_attn_bias): GraphormerGraphAttnBias(
        (edge_encoder): Embedding(1537, 32, padding_idx=0)
        (edge_dis_encoder): Embedding(131072, 1)
        (spatial_pos_encoder): Embedding(512, 32, padding_idx=0)
        (graph_token_virtual_distance): Embedding(1, 32)
      )
      (emb_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x GraphormerGraphEncoderLayer(
          (dropout_module): Dropout(p=0.0, inplace=False)
          (activation_dropout_module): Dr

In [21]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epoch = 10

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

loss_func = torch.nn.MSELoss()

/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
wandb.init(
    project="graphormer",
    name="Graphormer Simple Classification on MolecularWeight 10k 10_epochs",
    config={}
)

wandb: Currently logged in as: orlov-aleksei53 (moleculary-ai). Use `wandb login --relogin` to force relogin


### Training

In [26]:
print(loss)

tensor(97678.2344, device='cuda:4', grad_fn=<MseLossBackward0>)


In [25]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))

for epoch in range(num_epoch):
    model.train()
    train_epoch_loss = 0
    for batch in train_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() }
        
        outputs = model(**input_batch)
        print(outputs)
        loss = outputs["loss"]
        loss.backward()
        train_epoch_loss += loss.item()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    model.eval()
    eval_epoch_loss = 0
    for batch in eval_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() }

        with torch.no_grad():
            outputs = model(**input_batch)

        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        eval_epoch_loss += loss.item()
        progress_bar_eval.update(1)
    
    wandb.log({"loss/train": train_epoch_loss / len(train_dataloader), "loss/validation": eval_epoch_loss / len(eval_dataloader)})

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

SequenceClassifierOutput(loss=tensor(58840.5781, device='cuda:4', grad_fn=<MseLossBackward0>), logits=tensor([[30.2935],
        [30.2615]], device='cuda:4', grad_fn=<CloneBackward0>), hidden_states=[tensor([[[-9.8578e-04,  1.3590e-02,  1.9202e-03,  ..., -2.1354e-04,
          -5.9242e-04,  4.5683e-04],
         [-9.8578e-04,  1.3590e-02,  1.9202e-03,  ..., -2.1354e-04,
          -5.9242e-04,  4.5683e-04]],

        [[ 4.8131e-03,  2.7371e-01,  4.6840e-04,  ..., -1.6020e-03,
          -1.1349e-03,  1.6309e-02],
         [ 9.4549e-03, -4.1322e+00,  1.1811e-04,  ..., -2.3762e-03,
          -1.5545e-03,  2.4801e-02]],

        [[-8.5478e-03, -3.4004e+00,  2.4719e-03,  ...,  7.1614e-04,
           5.9348e-03,  5.0574e-02],
         [ 8.9118e-03, -1.6857e+00,  1.8907e-04,  ..., -1.9684e-03,
          -4.7803e-04,  2.5967e-02]],

        ...,

        [[-1.0699e-03,  5.5095e-02,  1.9580e-03,  ..., -8.2699e-05,
          -5.1617e-04,  9.0421e-02],
         [ 3.6241e-03,  2.9982e+00,  6.1201e-

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f0bb3fdc190>>
Traceback (most recent call last):
  File "/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/wandb/sdk/lib/redirect.py", line 648, in write
    cb(data)
  File "/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 2304, in <lambda>
    lambda data: self._console_raw_callback("stderr", data),
  File "/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 400, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/home/alexeyorlov

SequenceClassifierOutput(loss=tensor(97678.2344, device='cuda:4', grad_fn=<MseLossBackward0>), logits=tensor([[31.0584],
        [31.0052]], device='cuda:4', grad_fn=<CloneBackward0>), hidden_states=[tensor([[[-9.8569e-04,  1.3623e-02,  1.9182e-03,  ..., -2.1272e-04,
          -5.9413e-04,  4.5779e-04],
         [-9.8569e-04,  1.3623e-02,  1.9182e-03,  ..., -2.1272e-04,
          -5.9413e-04,  4.5779e-04]],

        [[-2.7785e-03, -4.4942e-01,  1.1903e-03,  ..., -1.0219e-03,
           2.1788e-03,  4.1401e-02],
         [ 8.9106e-03, -1.6856e+00,  1.8983e-04,  ..., -1.9667e-03,
          -4.7949e-04,  2.5969e-02]],

        [[ 1.8258e-03,  3.3817e+00,  6.2774e-04,  ..., -1.1692e-03,
           8.5251e-04,  2.7741e-02],
         [ 8.9106e-03, -1.6856e+00,  1.8983e-04,  ..., -1.9667e-03,
          -4.7949e-04,  2.5969e-02]],

        ...,

        [[ 8.9106e-03, -1.6856e+00,  1.8983e-04,  ..., -1.9667e-03,
          -4.7949e-04,  2.5969e-02],
         [-1.0698e-03,  5.5097e-02,  1.9560e-

KeyboardInterrupt: 

In [ ]:
# torch.onnx.export(model, input_batch, model_name_base + "_10k_10_epochs.onnx")
# wandb.save(model_name_base + "_10k_10_epochs.onnx")

## Post Training Evaluation

In [ ]:
test_dataloader = DataLoader(
    dataset_processed['test'], batch_size = batch_size, collate_fn = data_collator
)

model.eval()
epoch_loss = 0
for batch in tqdm(test_dataloader):
        input_batch = { k: v.to(device) for k, v in batch.items() }

        with torch.no_grad():
            outputs = model(**input_batch)

        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        epoch_loss += loss.item()
    
wandb.log({"loss/test": epoch_loss / len(test_dataloader)})

In [ ]:
wandb.finish()

In [ ]:
model.save_pretrained(model_name_base + '_10k_10epochs')

In [ ]:
# torch.save(model, model_name_base + '_10k_10epochs.pt')

In [ ]:
torch.cuda.empty_cache()